# 3.4 Calling differentially expressed peaks with DESeq2 and limma

### IMPORTANT: Please make sure that you are using the R kernel to run this notebook. ###


In this tutorial, we will focus on calling differential peaks: 
![Analysis pipeline](images/part4.png)

## Running DESeq

DESeq(https://bioconductor.org/packages/release/bioc/html/DESeq2.html) uses read count data, such as in our matrix **all.readcount.txt**, to estimate differential gene expression across conditions specified in a metdata file.  We run DESeq with 4 comparisons (which we call "contrasts"): 
* Timepoint 
    * 0hr vs 4hr
* Strain: 
    *  WT vs TF1
    *  WT vs TF2
    *  WT vs TF3
    *  WT vs TF4
    *  WT vs TF5
    *  WT vs TF6
    *  WT vs TF7
    *  WT vs TF8
    *  WT vs TF9   

In [ ]:
#change to your working directory 
username="annashch"
setwd(paste("/scratch/",username,sep=""))

In [ ]:
#load the DESeq2 library
library(DESeq2,quietly = TRUE)


In [ ]:
#We read in the counts data matrix and the metdata matrix in the same manner as we did in tutorial 3.1 
#load the read count matrix
count_data=read.table("/outputs/all.readcount.txt",header=TRUE)
rownames(count_data)=paste(count_data$Chrom,count_data$Start,count_data$End,sep='\t')
#remove the columns we will not use 
count_data$Chrom=NULL
count_data$Start=NULL
count_data$End=NULL
count_data$ID=NULL

head(count_data)

In [ ]:
metadata=read.table("/metadata/TC2017_samples.tsv",header=TRUE)
#We use the "factor" function to tell R which variables are categorical rather than continuous 
metadata$Strain=factor(metadata$Strain)
metadata$Timepoint=factor(metadata$Timepoint)
metadata$Researcher=factor(metadata$Researcher)
#we don't need the other metadata columns for this analysis 
metadata$Sample=NULL
metadata$Replicate=NULL
rownames(metadata)=metadata$ID
metadata$ID=NULL
#make sure the rows in metadata match the order of the columns in count_data 
metadata=metadata[names(count_data),]
head(metadata)

In [ ]:
#We set threshold for determining differential expression 
padjust_thresh=0.01 


In [ ]:
#create a DESeq2 object with the data, metadata, and model information 
ddsMat=DESeqDataSetFromMatrix(countData=as.matrix(count_data),
                            colData=metadata,
                            design=~Strain+Timepoint+Researcher)


In [ ]:
#Run DESeq2 analysis 
dds<-DESeq(ddsMat)

In [ ]:
#We can examine several contrasts in the resulting DESeq2 object
resultsNames(dds)

In [ ]:
#Specify the contrasts we want to examine (we indicated these above)
deseq_contrasts=list(c("Timepoint","0h","4h"),
                     c("Strain","WT","TF1"),
                     c("Strain","WT","TF2"),
                     c("Strain","WT","TF3"),
                     c("Strain","WT","TF4"),
                     c("Strain","WT","TF5"),
                     c("Strain","WT","TF6"),
                     c("Strain","WT","TF7"),
                     c("Strain","WT","TF8"),
                     c("Strain","WT","TF9"))
contrast_names=c("Timepoint_0h_vs_4h",
        "Strain_WT_vs_TF1",
        "Strain_WT_vs_TF2",
        "Strain_WT_vs_TF3",
        "Strain_WT_vs_TF4",
        "Strain_WT_vs_TF5",
        "Strain_WT_vs_TF6",
        "Strain_WT_vs_TF7",
        "Strain_WT_vs_TF8",
        "Strain_WT_vs_TF9")



In [ ]:
#Query the DESeq2 results to find differential peaks for each contrast, using our padjust_thresh and lfc_thresh values.
for(contrast_index in seq(1,10))
{
        comparison_name=unlist(contrast_names[contrast_index])    
        print(comparison_name)
        ds=results(dds,
           contrast=unlist(deseq_contrasts[contrast_index]))
       
        #write  entries for all peaks
        write.table(ds,file=paste(comparison_name,".txt",sep=""),quote=FALSE,row.names=TRUE,col.names=TRUE,sep='\t')
    
        #subset the peak set to just the differential peaks 
        ds=na.omit(ds)
        sig=ds[ds$padj<padjust_thresh,] 
    
        #find positive log fold change peaks 
        positive_sig=sig[sig$log2FoldChange > 0,]
    
        #find negative log fold change peaks 
        negative_sig=sig[sig$log2FoldChange <0,]
    
        write.table(positive_sig,
                    file=paste(comparison_name,".differential.positive.txt",sep=""),
                    quote=FALSE,row.names=FALSE,col.names=FALSE,sep='\t')
        write.table(negative_sig,
                    file=paste(comparison_name,".differential.negative.txt",sep=""),
                    quote=FALSE,row.names=FALSE,col.names=FALSE,sep='\t')
}


This code will generate 10 sets of files: 

* Timepoint_0h_vs_4h.txt  
* Timepoint_0h_vs_4h.differential.positive.txt  
* Timepoint_0h_vs_4h.differential.negative.txt  


* Strain_WT_vs_TF1.txt  
* Strain_WT_vs_TF1.differential.positive.txt
* Strain_WT_vs_TF1.differential.negative.txt


* Strain_WT_vs_TF2.txt  
* Strain_WT_vs_TF2.differential.positive.txt
* Strain_WT_vs_TF2.differential.negative.txt


* Strain_WT_vs_TF3.txt  
* Strain_WT_vs_TF3.differential.positive.txt
* Strain_WT_vs_TF3.differential.negative.txt


* Strain_WT_vs_TF4.txt  
* Strain_WT_vs_TF4.differential.positive.txt
* Strain_WT_vs_TF4.differential.negative.txt


* Strain_WT_vs_TF5.txt  
* Strain_WT_vs_TF5.differential.positive.txt
* Strain_WT_vs_TF5.differential.negative.txt


* Strain_WT_vs_TF6.txt  
* Strain_WT_vs_TF6.differential.positive.txt
* Strain_WT_vs_TF6.differential.negative.txt


* Strain_WT_vs_TF7.txt  
* Strain_WT_vs_TF7.differential.positive.txt
* Strain_WT_vs_TF7.differential.negative.txt


* Strain_WT_vs_TF8.txt  
* Strain_WT_vs_TF8.differential.positive.txt
* Strain_WT_vs_TF8.differential.negative.txt


* Strain_WT_vs_TF9.txt  
* Strain_WT_vs_TF9.differential.positive.txt
* Strain_WT_vs_TF9.differential.negative.txt


The first is the raw output from DESeq for all peaks. We will not have time to discuss everything in this file, but feel free to read the DESeq manual and see if you can understand it. The second,  contains a list of the IDs of the differentially open peaks from ATAC‐seq. The p‐value cutoff for differential openness that we use is 0.01. 

### Running limma ###

If you recall, we used the R limma package to remove the "Researcher" batch effect in our data. Limma can also be used for differential peak calling. Limma uses a similar algorithm to DESeq2. We will go through the process of calling differential peaks with limma and see how the peak rankings differ between limma and DESeq2 -- it's always best to sanity check your results by running them through several similar analysis algorithms. 

In [ ]:
#import the limma library 

library(limma)
#design the model 
design=model.matrix(~0+Strain+Timepoint+Researcher,data=metadata)

#We use the "voom" function associated with the limma package to normalize the count data 
vm=voom(count_data,design)

#fit the model to the data 
fit=lmFit(vm,design=vm$design)


#We'll examine the Timepoint contrast 
cont.matrix=makeContrasts(timepoint="Timepoint4h",levels=fit)
media_model=eBayes(contrasts.fit(fit,cont.matrix))
res_limma=topTable(media_model,n=nrow(count_data))
head(res_limma)

### Comparing DESeq2 and limma voom outputs ### 

In [ ]:
#Let's extract the media comparison from DESeq2
res_deseq2=results(dds,
           contrast=unlist(deseq_contrasts[1]))
res_deseq2=as.data.frame(res_deseq2)


In [ ]:
#We need to merge the two result dataframes by peak name So that we can generate a scatterplot of
#padj in one vs the other 
res_limma$peak=rownames(res_limma)
res_deseq2$peak=rownames(res_deseq2)
nrow(res_limma)
nrow(res_deseq2)

In [ ]:
merged_df=merge(res_limma,res_deseq2,by="peak")
merged_df$limma_padj=-10*log10(merged_df$padj)
merged_df$deseq2_padj=-10*log10(merged_df$adj.P.Val)



In [ ]:
head(merged_df)

In [ ]:
library(ggplot2)
ggplot(merged_df,aes(x=deseq2_padj,y=limma_padj))+
    geom_point(alpha=0.1)+
    xlim(0,400)+
    ylim(0,400)

The p-values appear to be pretty correlated. Let's make sure by computing the spearman and pearson correlations: 

In [ ]:
spearman_cor=cor(merged_df$limma_padj,merged_df$deseq2_padj,method="spearman")
spearman_cor

In [ ]:
pearson_cor=cor(merged_df$limma_padj,merged_df$deseq2_padj,method="pearson")
pearson_cor

Finally, we plot the rank comparison of the p-values across the two methods. 

In [ ]:
#use the "rank" function to generate rank columns for the p-values 
merged_df$limma_padj_rank=rank(merged_df$limma_padj)
merged_df$deseq2_padj_rank=rank(merged_df$deseq2_padj)

ggplot(merged_df,aes(x=deseq2_padj_rank,y=limma_padj_rank))+
    geom_point(alpha=0.1)